In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
# from d2lzh_pytorch import *

num_inputs = 2
num_examples = 10000
true_w = [2, -3.4]
true_b = 4.2
features = torch.randn(num_examples, num_inputs, dtype = torch.float32).view(-1, 2) # 确保是矩阵
labels = features[:, 0] * true_w[0] + features[:, 1] * true_w[1] + true_b # tensor具有广播机制, torch.Size([1000])
labels += torch.tensor(np.random.normal(0, 0.01, size = labels.size()), dtype = torch.float32)

# 一定要保证所有元素是矩阵
labels = labels.view(-1, 1) # 这一步很关键，我当时就是找bug这里出现了问题

In [2]:
def data_iter(batch_size, features, labels):
    number_examples = len(features)
    
    # shuffle
    indices = np.random.permutation(number_examples)
    
    for i in range(0, number_examples, batch_size):
        slice = indices[i : num_examples] if (i + batch_size > num_examples) else indices[i : i + batch_size]
        yield features[slice], labels[slice]

# test
for x, y in data_iter(batch_size = 10, features = features, labels = labels):
    print(x)
    print(y)
    break

tensor([[ 1.6934,  0.9817],
        [-0.2972,  1.8460],
        [ 1.1168, -1.1542],
        [-0.8056,  2.8604],
        [ 0.4212,  0.1680],
        [-0.7782,  0.0159],
        [-0.2379, -2.1238],
        [-1.1629, -2.3002],
        [ 0.9462, -1.7461],
        [-1.6638, -1.5105]])
tensor([[ 4.2500],
        [-2.6710],
        [10.3481],
        [-7.1333],
        [ 4.4642],
        [ 2.5931],
        [10.9495],
        [ 9.7045],
        [12.0411],
        [ 6.0269]])


模型初始化

In [3]:
w = torch.tensor(np.random.normal(0, 0.01, size = (num_inputs, 1)), dtype = torch.float32, requires_grad = True)
b = torch.tensor(1, dtype = torch.float32, requires_grad = True)

读取数据

定义模型

In [4]:
def linreg(x, w, b):
    return torch.matmul(x, w) + b

定义损失函数

In [5]:
# 返回的可能是标量，也可能是向量
def squared_loss(y, y_t):
    return (y - y_t) ** 2 / 2

定义sgd

In [6]:
# 因为这里求得的梯度是所有batch的梯度之和，所以要除以一个batch大小
def sgd(params, lr, batch_size):
    for param in params:
#         print(param.grad)
        param.data -= (lr * param.grad / batch_size) # 梯度更新时w和b不参与反向传播，要用.data

开始训练

In [7]:
epoch_number = 5
lr = 0.1
batchs_size = 50

for i in range(epoch_number):
    for data, label in data_iter(batchs_size, features, labels):
        model_res = linreg(data, w, b)
        loss = squared_loss(label, model_res).sum()
        loss.backward()
        sgd([w, b], lr, batchs_size)
        
        # 记得清零
        w.grad.data.zero_()
        b.grad.data.zero_()

    
    # 计算当前epoch的loss
    loss_current_epoch = squared_loss(labels, linreg(features, w, b)).mean().item()
    print('epoch = ' + str(i) + ', loss = ' + str(loss_current_epoch))

epoch = 0, loss = 5.0155489589087665e-05
epoch = 1, loss = 5.0383026973577216e-05
epoch = 2, loss = 5.023623089073226e-05
epoch = 3, loss = 5.009843880543485e-05
epoch = 4, loss = 5.016545037506148e-05


In [8]:
print(true_w)
print(w)
print('*' * 50)
print(true_b)
print(b)

[2, -3.4]
tensor([[ 2.0005],
        [-3.3998]], requires_grad=True)
**************************************************
4.2
tensor(4.1999, requires_grad=True)
